# Comparative analysis of LoRA fine-tuned BERT and DistilBERT for detecting AI-generated texts

In [1]:
%pip install -q transformers datasets evaluate peft torch pandas numpy scikit-learn

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd '/content/drive/MyDrive/Detecting AI-Generated Texts with DistilBERT'

/content/drive/.shortcut-targets-by-id/1XF6kskzIHpH3FrBSXKgdzgXIK2u413be/Detecting AI-Generated Texts with DistilBERT


## Load the data

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from datasets import Dataset, DatasetDict

In [5]:
# Read the CSV file into a DataFrame
df = pd.read_csv('essay_data.csv')

# Rename the columns
df = df.set_axis(['text', 'labels'], axis='columns')

# Display the first 5 rows of the DataFrame
print(df.head())

                                                text  labels
0  Car-free cities have become a subject of incre...       1
1  Car Free Cities  Car-free cities, a concept ga...       1
2    A Sustainable Urban Future  Car-free cities ...       1
3    Pioneering Sustainable Urban Living  In an e...       1
4    The Path to Sustainable Urban Living  In an ...       1


In [6]:
# Check the class distribution
class_counts = df['labels'].value_counts()
print(class_counts)

# Find the minimum class count
min_class_count = class_counts.min()

# Perform undersampling
undersampled_df = df.groupby('labels').apply(lambda x: x.sample(min_class_count)).reset_index(drop=True)

# Display the new class distribution
print(undersampled_df['labels'].value_counts())

# Display the first 5 rows of the undersampled DataFrame
print(undersampled_df.head())

labels
0    17508
1    11637
Name: count, dtype: int64
labels
0    11637
1    11637
Name: count, dtype: int64
                                                text  labels
0  Dear,Florida I think that we should keep the E...       0
1  There are so many reason why using a car can d...       0
2  The Electoral College should be abolished. It ...       0
3  ÃÂ¨95% of all people who ask for advice from ...       0
4  Do you really want to visit Venus? Because I s...       0


In [7]:
# Split the DataFrame into training and testing sets
# 20% of the data will be used for testing, and a random seed is set for reproducibility
train_df, test_df = train_test_split(undersampled_df, test_size=0.3, random_state=202406)

# Create a DatasetDict with training and validation datasets
# The training and validation sets are created from the respective DataFrames
dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df.reset_index(drop=True)),  # Reset index to ensure it's in sequential order
    "validation": Dataset.from_pandas(test_df.reset_index(drop=True))  # Reset index to ensure it's in sequential order
})

# Output the created dataset dictionary to check its structure
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 16291
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 6983
    })
})

## Create the model

In [8]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

In [9]:
# Define the checkpoint for the BERT model
check_point_bert = "bert-base-uncased"

# Define the checkpoint for the DistilBERT model
check_point_distilbert = "distilbert-base-uncased"

# Map of class ids to their corresponding labels
id2label = {0: "Human", 1: "AI"}

# Map of labels to their corresponding class ids
label2id = {"Human": 0, "AI": 1}

# Load the pre-trained BERT model for sequence classification with specified parameters
bert_model = AutoModelForSequenceClassification.from_pretrained(
    check_point_bert, num_labels=2, id2label=id2label, label2id=label2id, force_download=True, trust_remote_code=True
)

# Load the pre-trained DistilBERT model for sequence classification with specified parameters
distilbert_model = AutoModelForSequenceClassification.from_pretrained(
    check_point_distilbert, num_labels=2, id2label=id2label, label2id=label2id, force_download=True, trust_remote_code=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# View the model architecture of BERT
bert_model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [11]:
# View the model architecture of DistilBERT
distilbert_model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

## Preprocess the data

In [12]:
# Load the tokenizer for the BERT model from the specified checkpoint
# add_prefix_space=True ensures the tokenizer correctly handles word tokens with a leading space
# trust_remote_code=True allows loading the tokenizer even if it includes custom code
bert_tokenizer = AutoTokenizer.from_pretrained(
    check_point_bert,
    add_prefix_space=True,
    trust_remote_code=True
)

# Load the tokenizer for the DistilBERT model from the specified checkpoint
# add_prefix_space=True ensures the tokenizer correctly handles word tokens with a leading space
# trust_remote_code=True allows loading the tokenizer even if it includes custom code
distilbert_tokenizer = AutoTokenizer.from_pretrained(
    check_point_distilbert,
    add_prefix_space=True,
    trust_remote_code=True
)

In [13]:
# Check if the BERT tokenizer has a padding token
if bert_tokenizer.pad_token is None:
    # If not, add a special padding token "[PAD]"
    bert_tokenizer.add_special_tokens({"pad_token": "[PAD]"})
    # Resize the BERT model's token embeddings to include the new token
    bert_model.resize_token_embeddings(len(bert_tokenizer))

# Check if the DistilBERT tokenizer has a padding token
if distilbert_tokenizer.pad_token is None:
    # If not, add a special padding token "[PAD]"
    distilbert_tokenizer.add_special_tokens({"pad_token": "[PAD]"})
    # Resize the DistilBERT model's token embeddings to include the new token
    distilbert_model.resize_token_embeddings(len(distilbert_tokenizer))

In [14]:
def tokenize_bert(examples: dict) -> dict:
    """
    Tokenize input text using the BERT tokenizer.

    Args:
    examples (dict): A dictionary containing the text to be tokenized.

    Returns:
    dict: A dictionary containing tokenized inputs.
    """
    # Extract text from the examples dictionary
    text = examples["text"]

    # Set truncation side to 'left' for the BERT tokenizer
    bert_tokenizer.truncation_side = "left"

    # Tokenize the input text with truncation and return tensor format as numpy array
    tokenized_inputs = bert_tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

def tokenize_distilbert(examples: dict) -> dict:
    """
    Tokenize input text using the DistilBERT tokenizer.

    Args:
    examples (dict): A dictionary containing the text to be tokenized.

    Returns:
    dict: A dictionary containing tokenized inputs.
    """
    # Extract text from the examples dictionary
    text = examples["text"]

    # Set truncation side to 'left' for the DistilBERT tokenizer
    distilbert_tokenizer.truncation_side = "left"

    # Tokenize the input text with truncation and return tensor format as numpy array
    tokenized_inputs = distilbert_tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

In [15]:
# Tokenize the dataset using the BERT tokenizer
# Apply the tokenize_bert function to each example in the dataset in a batched manner
tokenized_dataset_bert = dataset.map(tokenize_bert, batched=True)

# Tokenize the dataset using the DistilBERT tokenizer
# Apply the tokenize_distilbert function to each example in the dataset in a batched manner
tokenized_dataset_distilbert = dataset.map(tokenize_distilbert, batched=True)

Map:   0%|          | 0/16291 [00:00<?, ? examples/s]

Map:   0%|          | 0/6983 [00:00<?, ? examples/s]

Map:   0%|          | 0/16291 [00:00<?, ? examples/s]

Map:   0%|          | 0/6983 [00:00<?, ? examples/s]

In [16]:
# Create a data collator that will dynamically pad the inputs received by the BERT tokenizer
bert_data_collator = DataCollatorWithPadding(tokenizer=bert_tokenizer)

# Create a data collator that will dynamically pad the inputs received by the DistilBERT tokenizer
distilbert_data_collator = DataCollatorWithPadding(tokenizer=distilbert_tokenizer)

## Setup evaluation metric

In [17]:
import evaluate

In [18]:
# Load the accuracy metric from the evaluate module
accuracy = evaluate.load("accuracy")

def compute_metrics(p):
    """
    Compute accuracy metric.

    Args:
    p (tuple): A tuple containing predictions and labels.

    Returns:
    dict: A dictionary containing the computed accuracy metric.
    """
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    # Compute accuracy using the loaded accuracy metric
    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

## Train the models

In [19]:
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig

In [20]:
# Define the configuration for the Lora model of BERT
peft_config_bert = LoraConfig(
    task_type="SEQ_CLS",    # Task type is sequence classification
    r=4,                    # Parameter r
    lora_alpha=32,          # Lora alpha value
    lora_dropout=0.01,      # Dropout rate for Lora
    target_modules=["query"] # Target modules for the model
)

# Define the configuration for the Lora model of DistilBERT
peft_config_dsitlbert = LoraConfig(
    task_type="SEQ_CLS",    # Task type is sequence classification
    r=4,                    # Parameter r
    lora_alpha=32,          # Lora alpha value
    lora_dropout=0.01,      # Dropout rate for Lora
    target_modules=["q_lin"] # Target modules for the model
)

In [21]:
# Modify the BERT model using the PEFT configuration
bert_model = get_peft_model(bert_model, peft_config_bert)

# Print the trainable parameters of the modified BERT model
bert_model.print_trainable_parameters()

trainable params: 75,266 || all params: 109,559,044 || trainable%: 0.0687


In [22]:
# Modify the DistilBERT model using the PEFT configuration
distilbert_model = get_peft_model(distilbert_model, peft_config_dsitlbert)

# Print the trainable parameters of the modified DistilBERT
distilbert_model.print_trainable_parameters()

trainable params: 628,994 || all params: 67,584,004 || trainable%: 0.9307


In [23]:
# Learning rate for training
lr = 1e-3

# Batch size used during training
batch_size = 8

# Number of epochs for training (generally 1 - 3 for fine-tuning)
num_epochs = 1

In [24]:
# Define training arguments for BERT model training
training_args_bert = TrainingArguments(
    output_dir=check_point_bert + "-lora-ai-generated-texts-detection",  # Directory to save model checkpoints
    learning_rate=lr,  # Learning rate for optimizer
    per_device_train_batch_size=batch_size,  # Batch size for training
    per_device_eval_batch_size=batch_size,  # Batch size for evaluation
    num_train_epochs=num_epochs,  # Number of training epochs
    weight_decay=0.01,  # Weight decay to apply
    evaluation_strategy="epoch",  # Evaluate every epoch
    save_strategy="epoch",  # Save model every epoch
    load_best_model_at_end=True,  # Load the best model at the end of training
)

# Define Trainer for BERT model training
trainer_bert = Trainer(
    model=bert_model,  # The BERT model to be trained
    args=training_args_bert,  # Training arguments
    train_dataset=tokenized_dataset_bert["train"],  # Training dataset
    eval_dataset=tokenized_dataset_bert["validation"],  # Evaluation dataset
    tokenizer=bert_tokenizer,  # Tokenizer for tokenizing input data
    data_collator=bert_data_collator,  # Data collator for batching and padding
    compute_metrics=compute_metrics,  # Function to compute evaluation metrics
)

# Train the BERT model
trainer_bert.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.017600,0.070862,{'accuracy': 0.986395532006301}


Trainer is attempting to log a value of "{'accuracy': 0.986395532006301}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TrainOutput(global_step=2037, training_loss=0.04231551018894216, metrics={'train_runtime': 1426.7602, 'train_samples_per_second': 11.418, 'train_steps_per_second': 1.428, 'total_flos': 4266144814694400.0, 'train_loss': 0.04231551018894216, 'epoch': 1.0})

In [25]:
# Define training arguments for DistilBERT model training
training_args_distilbert = TrainingArguments(
    output_dir=check_point_distilbert + "-lora-ai-generated-texts-detection",  # Directory to save model checkpoints
    learning_rate=lr,  # Learning rate for optimizer
    per_device_train_batch_size=batch_size,  # Batch size for training
    per_device_eval_batch_size=batch_size,  # Batch size for evaluation
    num_train_epochs=num_epochs,  # Number of training epochs
    weight_decay=0.01,  # Weight decay to apply
    evaluation_strategy="epoch",  # Evaluate every epoch
    save_strategy="epoch",  # Save model every epoch
    load_best_model_at_end=True,  # Load the best model at the end of training
)

# Define Trainer for DistilBERT model training
trainer_distilbert = Trainer(
    model=distilbert_model,  # The DistilBERT model to be trained
    args=training_args_distilbert,  # Training arguments
    train_dataset=tokenized_dataset_distilbert["train"],  # Training dataset
    eval_dataset=tokenized_dataset_distilbert["validation"],  # Evaluation dataset
    tokenizer=distilbert_tokenizer,  # Tokenizer for tokenizing input data
    data_collator=distilbert_data_collator,  # Data collator for batching and padding
    compute_metrics=compute_metrics,  # Function to compute evaluation metrics
)

# Train the DistilBERT model
trainer_distilbert.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.017500,0.040616,{'accuracy': 0.9898324502362881}


Trainer is attempting to log a value of "{'accuracy': 0.9898324502362881}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TrainOutput(global_step=2037, training_loss=0.045227328146212116, metrics={'train_runtime': 733.9683, 'train_samples_per_second': 22.196, 'train_steps_per_second': 2.775, 'total_flos': 2177274625228800.0, 'train_loss': 0.045227328146212116, 'epoch': 1.0})